In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm

# 数据路径
train_dir = '/root/datasets/BTXRD3/train'
val_dir = '/root/datasets/BTXRD3/val'

# 数据预处理和加载
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)
class_names = full_dataset.classes

# 构建DenseNet-121模型
def build_model():
    model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
    model.classifier = nn.Linear(model.classifier.in_features, 3)
    return model

# 交叉验证和模型训练
kf = KFold(n_splits=5, shuffle=True, random_state=42)
metrics = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

best_val_loss = float('inf')
best_model_path = 'best_model.pth'
best_epoch = -1
best_fold = -1

fold_val_true = []
fold_val_pred = []
fold_val_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(full_dataset)):
    print(f'Fold {fold + 1}')
    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
    
    model = build_model().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    fold_best_val_loss = float('inf')
    fold_best_epoch = -1
    fold_best_model_path = f'best_model_fold_{fold + 1}.pth'

    for epoch in range(20):
        model.train()
        train_loss = 0.0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        model.eval()
        val_loss = 0.0
        val_true = []
        val_pred = []
        val_scores = []
        with torch.no_grad():
            for images, labels in tqdm(val_loader):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                val_true.extend(labels.cpu().numpy())
                val_pred.extend(outputs.argmax(dim=1).cpu().numpy())
                val_scores.extend(torch.softmax(outputs, dim=1).cpu().numpy())
        
        val_loss /= len(val_loader.dataset)
        
        if val_loss < fold_best_val_loss:
            fold_best_val_loss = val_loss
            fold_best_epoch = epoch
            torch.save(model.state_dict(), fold_best_model_path)
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    if fold_best_val_loss < best_val_loss:
        best_val_loss = fold_best_val_loss
        best_epoch = fold_best_epoch
        best_fold = fold + 1
        torch.save(model.state_dict(), best_model_path)
    
    model.load_state_dict(torch.load(fold_best_model_path))
    val_true = np.array(val_true)
    val_pred = np.array(val_pred)
    val_scores = np.array(val_scores)

    fold_val_true.extend(val_true)
    fold_val_pred.extend(val_pred)
    fold_val_scores.extend(val_scores)
    
    accuracy = accuracy_score(val_true, val_pred)
    precision = precision_score(val_true, val_pred, average='weighted')
    recall = recall_score(val_true, val_pred, average='weighted')
    f1 = f1_score(val_true, val_pred, average='weighted')
    auc = roc_auc_score(val_true, val_scores, multi_class='ovo')
    
    print(f'Best model found at epoch {fold_best_epoch}')
    print(f'Val Loss: {fold_best_val_loss:.4f} Acc: {accuracy:.4f} Precision: {precision:.4f} Recall: {recall:.4f} F1: {f1:.4f} AUC: {auc:.4f}')
    
    class_metrics = {}
    for class_idx, class_name in enumerate(full_dataset.classes):
        class_true = (val_true == class_idx).astype(int)
        class_pred = (val_pred == class_idx).astype(int)
        class_score = val_scores[:, class_idx]
        
        class_accuracy = accuracy_score(class_true, class_pred)
        class_precision = precision_score(class_true, class_pred)
        class_recall = recall_score(class_true, class_pred)
        class_f1 = f1_score(class_true, class_pred)
        class_auc = roc_auc_score(class_true, class_score)
        
        class_metrics[class_name] = {
            'accuracy': class_accuracy,
            'precision': class_precision,
            'recall': class_recall,
            'f1': class_f1,
            'auc': class_auc
        }
        
        print(f'Val Class {class_name} Acc: {class_accuracy:.4f} Precision: {class_precision:.4f} Recall: {class_recall:.4f} F1: {class_f1:.4f} AUC: {class_auc:.4f}')
    
    metrics.append({'fold': fold + 1, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1, 'auc': auc, 'class_metrics': class_metrics, 'best_epoch': fold_best_epoch})

# 保存五折交叉验证的指标
metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv('cross_validation_metrics.csv', index=False)

# 绘制五折交叉验证的ROC曲线
plt.figure()
for class_idx, class_name in enumerate(full_dataset.classes):
    class_true = (np.array(fold_val_true) == class_idx).astype(int)
    class_score = np.array(fold_val_scores)[:, class_idx]
    
    fpr, tpr, _ = roc_curve(class_true, class_score)
    auc = roc_auc_score(class_true, class_score)
    
    plt.plot(fpr, tpr, label=f'{class_name} (area = {auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Cross-Validation ROC Curve')
plt.legend(loc="lower right")
plt.savefig('cross_validation_roc_curve.pdf')
plt.close()

# 绘制五折交叉验证的混淆矩阵
cm = confusion_matrix(fold_val_true, fold_val_pred)
plt.figure()
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Cross-Validation Confusion Matrix')
plt.savefig('cross_validation_confusion_matrix.pdf')
plt.close()

# 加载最终最优模型并评估独立验证集
model.load_state_dict(torch.load(best_model_path))
model.eval()

val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
val_true = []
val_pred = []
val_scores = []
with torch.no_grad():
    for images, labels in tqdm(val_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        val_true.extend(labels.cpu().numpy())
        val_pred.extend(outputs.argmax(dim=1).cpu().numpy())
        val_scores.extend(torch.softmax(outputs, dim=1).cpu().numpy())

val_true = np.array(val_true)
val_pred = np.array(val_pred)
val_scores = np.array(val_scores)

# 计算独立验证集的指标
accuracy = accuracy_score(val_true, val_pred)
precision = precision_score(val_true, val_pred, average='weighted')
recall = recall_score(val_true, val_pred, average='weighted')
f1 = f1_score(val_true, val_pred, average='weighted')
auc = roc_auc_score(val_true, val_scores, multi_class='ovo')

print(f'Validation Set Metrics:')
print(f'Acc: {accuracy:.4f} Precision: {precision:.4f} Recall: {recall:.4f} F1: {f1:.4f} AUC: {auc:.4f}')

# 保存独立验证集的指标
val_metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1,
    'auc': auc,
    'class_metrics': {}
}

for class_idx, class_name in enumerate(val_dataset.classes):
    class_true = (val_true == class_idx).astype(int)
    class_pred = (val_pred == class_idx).astype(int)
    class_score = val_scores[:, class_idx]
    
    class_accuracy = accuracy_score(class_true, class_pred)
    class_precision = precision_score(class_true, class_pred)
    class_recall = recall_score(class_true, class_pred)
    class_f1 = f1_score(class_true, class_pred)
    class_auc = roc_auc_score(class_true, class_score)
    
    val_metrics['class_metrics'][class_name] = {
        'accuracy': class_accuracy,
        'precision': class_precision,
        'recall': class_recall,
        'f1': class_f1,
        'auc': class_auc
    }
    
    print(f'Val Class {class_name} Acc: {class_accuracy:.4f} Precision: {class_precision:.4f} Recall: {class_recall:.4f} F1: {class_f1:.4f} AUC: {class_auc:.4f}')

# 保存独立验证集的指标
with open('validation_metrics.csv', 'w') as f:
    for key in val_metrics.keys():
        f.write(f"{key},{val_metrics[key]}\n")

# 绘制独立验证集的ROC曲线
plt.figure()
for class_idx, class_name in enumerate(val_dataset.classes):
    class_true = (val_true == class_idx).astype(int)
    class_score = val_scores[:, class_idx]
    
    fpr, tpr, _ = roc_curve(class_true, class_score)
    auc = roc_auc_score(class_true, class_score)
    
    plt.plot(fpr, tpr, label=f'{class_name} (area = {auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Validation ROC Curve')
plt.legend(loc="lower right")
plt.savefig('validation_roc_curve.pdf')
plt.close()

# 绘制独立验证集的混淆矩阵
cm = confusion_matrix(val_true, val_pred)
plt.figure()
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Validation Confusion Matrix')
plt.savefig('validation_confusion_matrix.pdf')
plt.close()

# 输出最终最优模型的折数和最佳轮次
print(f'The best model was found in fold {best_fold} at epoch {best_epoch}')


Fold 1


100%|██████████| 21/21 [00:18<00:00,  1.13it/s]


Epoch 1, Train Loss: 0.8124, Val Loss: 0.8995


100%|██████████| 21/21 [00:18<00:00,  1.16it/s]


Epoch 2, Train Loss: 0.7032, Val Loss: 0.8173


...

Epoch 19, Train Loss: 0.1921, Val Loss: 0.9685


100%|██████████| 21/21 [00:21<00:00,  1.03s/it]


Epoch 20, Train Loss: 0.1030, Val Loss: 1.0628
Best model found at epoch 2
Val Loss: 0.7283 Acc: 0.6930 Precision: 0.7163 Recall: 0.6930 F1: 0.6893 AUC: 0.8807
Val Class benign Acc: 0.7249 Precision: 0.5316 Recall: 0.6429 F1: 0.5820 AUC: 0.7837
Val Class malignant Acc: 0.8891 Precision: 0.7541 Recall: 0.9340 F1: 0.8345 AUC: 0.9618
Val Class normal Acc: 0.7720 Precision: 0.8249 Recall: 0.5509 F1: 0.6606 AUC: 0.8837


100%|██████████| 35/35 [00:40<00:00,  1.16s/it]


Validation Set Metrics:
Acc: 0.6392 Precision: 0.6935 Recall: 0.6392 F1: 0.6538 AUC: 0.8281
Val Class benign Acc: 0.7215 Precision: 0.6856 Recall: 0.5513 F1: 0.6111 AUC: 0.7636
Val Class malignant Acc: 0.8210 Precision: 0.3092 Recall: 0.7476 F1: 0.4375 AUC: 0.8517
Val Class normal Acc: 0.7360 Precision: 0.7698 Recall: 0.6879 F1: 0.7266 AUC: 0.8262
The best model was found in fold 3 at epoch 4
